In [180]:
import pandas as pd
from pyecharts.charts import WordCloud, Bar, Pie
from pyecharts import options as opts

In [160]:
file1=r'D:\Programing\python_projects\data_analysis_basics\data_set\rating.txt'
rating=pd.read_csv(file1, header=None, names=['user_id', 'book_id', 'rating'])
file2=r'D:\Programing\python_projects\data_analysis_basics\data_set\to_read.csv'
to_read=pd.read_csv(file2)
file3=r'D:\Programing\python_projects\data_analysis_basics\data_set\books.csv'
books=pd.read_csv(file3)
file4=r'D:\Programing\python_projects\data_analysis_basics\data_set\book_tags.csv'
book_tags=pd.read_csv(file4)
file5=r'D:\Programing\python_projects\data_analysis_basics\data_set\tags.csv'
tags=pd.read_csv(file5)

In [161]:
# 创建空DF存放书本平均得分，并指定索引名
book_df=pd.DataFrame({
    'rating_sum':[],
    'rating_counts':[]
})
book_df.index.name='book_id'

In [162]:
# 计算不同的书被评分的次数
book_df['rating_counts']=rating.groupby('book_id')['user_id'].count()

In [163]:
# 计算不同的书被评分的总和
book_df['rating_sum']=rating.groupby('book_id')['rating'].sum()
book_df.head()

,rating_sum,rating_counts
book_id,,
1,97603,22806
2,95077,21850
3,54422,16931
4,82639,19088
5,62634,16604


In [164]:
# 计算不同书的平均得分
book_df['avg_rating']=book_df['rating_sum']/book_df['rating_counts']
book_df.head()

,rating_sum,rating_counts,avg_rating
book_id,,,
1,97603,22806,4.279707
2,95077,21850,4.351350
3,54422,16931,3.214341
4,82639,19088,4.329369
5,62634,16604,3.772224


In [165]:
# 统计想读的书的人次并排序
to_read_counts=to_read['book_id'].value_counts().sort_values(ascending=False)
to_read_counts

47      2772
143     1967
113     1840
13      1812
11      1767
        ... 
9345       1
9803       1
3662       1
4434       1
9781       1
Name: book_id, Length: 9986, dtype: int64

In [166]:
# 截取前50的book id
top50_book_id=to_read_counts[:50].index
top50_book_counts=to_read_counts[:50].values
top50_books=pd.DataFrame({
    'book_id':top50_book_id,
    'to_read_count':top50_book_counts
})
top50_books.head(10)

,book_id,to_read_count
0,47,2772
1,143,1967
2,113,1840
3,13,1812
4,11,1767
5,45,1717
6,139,1650
7,39,1619
8,65,1608
9,35,1576


In [167]:
# 取得带有标题的DF
book_id_and_title=books[['book_id', 'goodreads_book_id', 'title']]
book_id_and_title.head()

,book_id,goodreads_book_id,title
0,1,2767052,"The Hunger Games (The Hunger Games, #1)"
1,2,3,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,41865,"Twilight (Twilight, #1)"
3,4,2657,To Kill a Mockingbird
4,5,4671,The Great Gatsby


In [168]:
# 合并DF，找到对应的名称
top50_books_with_title=pd.merge(top50_books, book_id_and_title, on='book_id', how='left')
top50_books_with_title.head(10)

,book_id,to_read_count,goodreads_book_id,title
0,47,2772,19063,The Book Thief
1,143,1967,18143977,All the Light We Cannot See
2,113,1840,168668,Catch-22
3,13,1812,5470,1984
4,11,1767,77203,The Kite Runner
5,45,1717,4214,Life of Pi
6,139,1650,9460487,Miss Peregrine’s Home for Peculiar Children (M...
7,39,1619,13496,"A Game of Thrones (A Song of Ice and Fire, #1)"
8,65,1608,4981,Slaughterhouse-Five
9,35,1576,865,The Alchemist


In [169]:
# 以CSV格式储存
top50_books_with_title.to_csv(r'D:\Programing\python_projects\data_analysis_basics\data_set\top50_books_with_title.csv')

In [170]:
# 通过关联字段找到最热门标签ID
book_tags=book_tags[book_tags['_goodreads_book_id_'].isin(top50_books_with_title['goodreads_book_id'])]
del book_tags['_goodreads_book_id_']
top10_tags=book_tags.groupby('tag_id').sum()
top10_tags=top10_tags.sort_values(by='count', ascending=False)[:10]
top10_tags

,count
tag_id,
30574,6061902
8717,412677
11557,404973
11743,299256
7457,279578
14487,80671
11305,71863
5207,71629
22743,65001


In [171]:
# 通过tag_id找到对应的名称
top10_tags_with_name=pd.merge(top10_tags, tags, on='tag_id', how='left')
top10_tags_with_name

,tag_id,count,tag_name
0,30574,6061902,to-read
1,8717,412677,currently-reading
2,11557,404973,favorites
3,11743,299256,fiction
4,7457,279578,classics
5,14487,80671,historical-fiction
6,11305,71863,fantasy
7,5207,71629,books-i-own
8,22743,65001,owned
9,33114,56829,young-adult


In [172]:
# 以CSV格式储存
top10_tags_with_name.to_csv(r'D:\Programing\python_projects\data_analysis_basics\data_set\top10_tags_with_name.csv')

In [176]:
# 绘制最想读的50本书的词云图
title=top50_books_with_title['title'].tolist()
to_read_count=top50_books_with_title['to_read_count'].tolist()
words=[list(z) for z in zip(title, to_read_count)]

wordcloud=WordCloud(init_opts=opts.InitOpts(width='1280px', height=('720px')))
wordcloud.add('', words, word_size_range=[20,100])
wordcloud.set_global_opts(title_opts=opts.TitleOpts(title='top 50 books to read'))
wordcloud.render('top50_books.html')

'D:\\Programing\\python_projects\\top50_books.html'

In [179]:
# 用柱状图显示10个热门标签数据
bar=Bar()
bar.add_xaxis(top10_tags_with_name['tag_name'].tolist())
bar.add_yaxis('tags', top10_tags_with_name['count'].tolist())
bar.set_global_opts(title_opts=opts.TitleOpts(title='top 10 tags'),
                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)))
bar.render('top10_tags.html')

'D:\\Programing\\python_projects\\top10_tags.html'

In [185]:
# 用饼图显示10个热门标签数据
tag_name=top10_tags_with_name['tag_name']
count=top10_tags_with_name['count']/1000
data=[list(z) for z in zip(tag_name, count)]

pie=Pie()
pie.add('', data, radius=['20%', '70%'])
pie.set_global_opts(title_opts=opts.TitleOpts(title='top 10 tags'),
                    legend_opts=opts.LegendOpts(is_show=False))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{c}'))  # 标签显示格式
pie.render(path='top10_tags_pie.html')


'D:\\Programing\\python_projects\\top10_tags_pie.html'